In [8]:
ctrl_svr_ip = "129.114.108.5"

svr_ips = [
    "10.52.3.84",
    "10.52.3.34",
    "10.52.3.164",
    "",
    "10.52.0.30",
    "",
    "10.52.3.162",
    "10.52.0.170",
    "10.52.0.58",
    "10.52.2.100",
    "",
    "",
    "",
    "10.52.2.115",
    "",
    "",
    "10.52.3.4",
    "10.52.2.188",
    "10.52.2.75",
    "10.52.3.26",
    "",
    "",
    "",
    "10.52.0.34",
    "10.52.2.47",
    "10.52.3.118",
    "10.52.2.198",
    "",
    "10.52.3.58",
    "10.52.2.86",
    "",
    "10.52.2.151",
    "",
    "10.52.2.123",
    # "10.52.1.100", # remove this ip as it has no connection
    "localhost"
]

In [ ]:
import paramiko
from concurrent.futures import ThreadPoolExecutor

pkey = paramiko.RSAKey.from_private_key_file("/Users/ybyan/.ssh/ddb-access.pem")

# Jump host details
jump_host = ctrl_svr_ip
jump_user = "cc"

# Worker node details
worker_ips = [ ip for ip in svr_ips if ip ]
worker_user = "cc"

workers = []
ctrl = None

try:
    # Connect to the jump server
    ctrl = paramiko.SSHClient()
    ctrl.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ctrl.connect(jump_host, username=jump_user, pkey=pkey)
except Exception as e:
    print(f"Controller server {jump_host} failed with error: {e}")

# for w_ip in worker_ips:
def connect_worker(w_ip):
    try:
        # Open a channel to the worker node
        jump_transport = ctrl.get_transport()
        dest_addr = (w_ip, 22)
        local_addr = (jump_host, 22)
        worker_channel = jump_transport.open_channel("direct-tcpip", dest_addr, local_addr)

        # Connect to the worker node via the channel
        worker_client = paramiko.SSHClient()
        worker_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        worker_client.connect(
            w_ip,
            username=worker_user,
            pkey=pkey,
            sock=worker_channel
        )

        return worker_client
    except Exception as e:
        print(f"Worker {w_ip} failed with error: {e}")
        return None

with ThreadPoolExecutor(max_workers=len(worker_ips)) as executor:
    # futures = [executor.submit(execute_command_on_worker, ip) for ip in worker_ips]
    futures = [ 
        executor.submit(connect_worker, ip) for ip in worker_ips
    ]

    # Gather and print results
    for future in futures:
        # print(future.result())
        if future.result():
            workers.append(future.result())

Worker 10.52.2.100 failed with error: ChannelException(2, 'Connect failed')


In [28]:
def execute_command_on_worker(worker: paramiko.SSHClient, command: str = "hostname") -> str:
    """Connect to a worker node via the jump server and execute a command."""
    try:
        # Execute the command on the worker node
        stdin, stdout, stderr = worker.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()

        return f"{output}{error}"
    except Exception as e:
        return f"Worker {worker} failed with error: {e}"

# Run the workflow on all workers in parallel
def exec_cmd(command: str, silent: bool = False):
    with ThreadPoolExecutor(max_workers=len(worker_ips)) as executor:
        futures = [
            executor.submit(execute_command_on_worker, w, command) 
            for w in workers
        ]

        # Gather and print results
        for future in futures:
            r = future.result()
            if not silent:
                print(r)

In [ ]:
# install uv for python environment on coordinator server.
output = ctrl.exec_command("curl -LsSf https://astral.sh/uv/install.sh | sh")[1].read().decode()
print(output)

no checksums to verify
installing to /home/cc/.local/bin
  uv
  uvx
everything's installed!

To add $HOME/.local/bin to your PATH, either restart your shell or run:

    source $HOME/.local/bin/env (sh, bash, zsh)
    source $HOME/.local/bin/env.fish (fish)



In [ ]:
# assert(exec_cmd("ls /shared"))

In [18]:
exec_cmd("sudo apt-get update; sudo apt-get install -y gdb build-essential cmake")

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:2 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Get:5 http://security.ubuntu.com/ubuntu noble-security/main amd64 Components [7184 B]
Get:6 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Components [51.8 kB]
Get:7 http://security.ubuntu.com/ubuntu noble-security/restricted amd64 Components [208 B]
Get:8 http://security.ubuntu.com/ubuntu noble-security/multiverse amd64 Components [208 B]
Fetched 441 kB in 1s (520 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.10ubuntu1).
The following additional packages will be installed:
  cmake-data libbabeltrace1 libdebuginfod-common libdebuginfod1t64 libipt2
  libjsonc

In [ ]:
# set up ssh key for all servers
stdin, stdout, stderr = ctrl.exec_command('yes y | ssh-keygen -t rsa -b 2048 -f ~/.ssh/id_rsa -q -N ""')
output = stdout.read().decode()
error = stderr.read().decode()

if error:
    print(f"Error: failed to gen ssh key: {error}")
else:
    _, stdout, stderr = ctrl.exec_command("cat ~/.ssh/id_rsa.pub | tee -a ~/.ssh/authorized_keys")
    output = stdout.read().decode()
    error = stderr.read().decode()
    if error:
        print(f"Error: failed to add ssh key to authorized_keys: {error}")
    else:
        print(f"Adding ssh key: {output}")
        exec_cmd(f"echo '{output.strip()}' >> ~/.ssh/authorized_keys", silent=True)

Adding ssh key: ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCMSfpnB0E997gqQNh6UV7uPXGBLpAh/Hp5PsSPYCpbs1r5RxoyGTrw1SYl8r+w4TMvTFsOPxnbDF1DYDrhp/G9aZzNttT+6GoQVKK42QDn1/Omdd9ujdijBWydguRgjOfLrSleG04CmLK5XE3mvMxLZuHGjE8US2lDXzWztaHheegXd1Gc140AFbP0QtgU5DRMnpRCGY78doZKCvcoX7PFZfMp3MoMrrFR2TGGxYtdrB31dmS9WYU3NcMyV2RnD+jHwZPXkooRZ9DQ9MH7c/dSK586VkaF/+nwJnjJu1bsag0U2igXRuwdiTI+aKoSYXFA4fNPlTH6pjOsxAMKzwY1 cc@node



In [19]:
exec_cmd("cd /shared/distributed-debugger && make setup-machine")

cd ./scripts && ./setup_machine.sh
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [131 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Components [309 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/restricted amd64 Components [212 B]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/multiverse amd64 Components [940 B]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports/main amd64 Components [208 B]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports/universe amd64 Components [11.7 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu nob

In [ ]:
# exec_cmd("cd /shared/distributed-debugger && make gdb-config-setup")

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

cd ./scripts && ./setup_gdb.sh
0

